Camel 3 Function 
=====================


In [ ]:
include(joinpath(dirname(Base.find_package("Globtim")), "..", ".globtim", "notebook_setup.jl"))

We define the center and range of the square domain of approximation. 

In [ ]:
# Constants and Parameters
const n, a, b = 2, 50, 1
const scale_factor = a / b
f = dejong5 # Objective function
d = 24 # Initial Degree 
SMPL = 200   
println("Number of samples: ", SMPL^2)
TR = test_input(f, 
                dim = n,
                center = [0.0, 0.0],
                GN = SMPL, 
                sample_range = scale_factor
                )

The `Constructor` function computes the coefficients of the polynomial approximant in the basis specified, with `GN^2` samples distributed following the measure with respect to which the polynomial basis is orthogonal.
The `RationalPrecision` specifies that the conversion of the polynomial's coefficients from the tensorized orthogonal basis to the standard monomial basis is carried out in high precision. 

In [ ]:
pol_cheb = Constructor(TR, d, basis=:chebyshev, precision=RationalPrecision)
pol_lege = Constructor(TR, d, basis=:legendre, precision=RationalPrecision, normalized=true);

Note that with the `Legendre` polynomials, it is necessary to run with the `normalized` option to ensure that the polynomials are orthonormal with respect to the uniform measure on the domain of approximation.

In [ ]:
@polyvar(x[1:n]) # Define polynomial ring 
real_pts_cheb = solve_polynomial_system(
    x, n, d, pol_cheb.coeffs;
    basis=pol_cheb.basis,
    precision=pol_cheb.precision,
    normalized=pol_cheb.normalized,
)

real_pts_lege = solve_polynomial_system(
    x, n, d, pol_lege.coeffs;
    basis=pol_lege.basis,
    precision=pol_lege.precision,
    normalized=pol_lege.normalized)

df_cheb = process_crit_pts(real_pts_cheb, f, TR)
df_lege = process_crit_pts(real_pts_lege, f, TR);

In [ ]:
using Optim
df_cheb, df_min_cheb = analyze_critical_points(f, df_cheb, TR, tol_dist=.5);
df_lege, df_min_lege = analyze_critical_points(f, df_lege, TR, tol_dist=.5);

In [ ]:
fig_1 = cairo_plot_polyapprox_levelset(pol_cheb, TR, df_cheb, df_min_cheb, chebyshev_levels=true, figure_size = (800, 800))
display(fig_1)

In [ ]:
# fig_2 = cairo_plot_polyapprox_levelset(pol_lege, TR, df_lege, df_min_lege, chebyshev_levels=true, figure_size = (800, 800))
# display(fig_2)

In [ ]:
using GLMakie
GLMakie.activate!()  # Ensure GLMakie is active for 3D plots

In [ ]:
fig_3d = plot_polyapprox_3d(
    pol_lege, 
    TR, 
    df_lege, 
    df_min_lege, 
    figure_size=(1600, 1600),
    fade=true,
    z_cut=0.001
)
display(fig_3d)

In [ ]:
# Commented out save command
# save("DeJong_lege_3d.png", fig_3d)